In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
from warnings import warn
import shutil
from custom_steps import (
    step_resnet18_tidy,
    step_resnet18_streamline,
    step_resnet18_convert_to_hls,
    step_resnet18_set_fifo_depths,
    step_resnet18_slr_floorplan)

model_file = "models/resnet18_img_3.onnx"
board = "KV260_SOM"
additional = "100-FPS"
if additional != "":
    final_output_dir = "build-{}/{}-{}".format(board,model_file.split('.')[0].split('/')[1],additional)
    os.environ["FINN_BUILD_DIR"] = "/workspace/results/%s/%s-%s" %(board,model_file.split('.')[0].split('/')[1],additional)
    inter_build_dir = os.environ["FINN_BUILD_DIR"]
else:
    final_output_dir = "build-{}/{}".format(board,model_file.split('.')[0].split('/')[1])

    os.environ["FINN_BUILD_DIR"] = "/workspace/results/%s/%s" %(board,model_file.split('.')[0].split('/')[1])
    inter_build_dir = os.environ["FINN_BUILD_DIR"]

if os.path.exists(inter_build_dir):
    shutil.rmtree(inter_build_dir)
    print("Previous intermeditate build deleted!")

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")
    
resnet18_build_steps = [
    step_resnet18_tidy,
    step_resnet18_streamline,
    step_resnet18_convert_to_hls,
    "step_create_dataflow_partition",
    "step_apply_folding_config",
    "step_generate_estimate_reports",
    "step_hls_codegen",
    "step_hls_ipgen",
    step_resnet18_set_fifo_depths,
    step_resnet18_slr_floorplan,
    "step_synthesize_bitfile",
    "step_make_pynq_driver",
    "step_deployment_package",
]

try:
    from finn.transformation.fpgadataflow.infer_doublepacked_dsp import InferDoublePackedConv
    folding_config_file="folding_config/U250_folding_config.json"
    print("DoublePackedConv detected")
except:
    warn(" FINN Experimental not available. Using non-packed folded down convolution. This is 16 times slower per MHz ")
    folding_config_file="folding_config/U250_folding_config_no_doublepack_pe_folded_16.json"
# folding_config_file = "folding_config/KV260_folding_config_no_doublepack_pe_folded_16.json"

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    steps               = resnet18_build_steps,
    mvau_wwidth_max     = 80,
    target_fps          = 100,
    synth_clk_period_ns = 5.0,
    folding_config_file = folding_config_file,
    board               = board,
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

<ipython-input-1-7d4986b03205>:56: UserWarning:  FINN Experimental not available. Using non-packed folded down convolution. This is 16 times slower per MHz 
  warn(" FINN Experimental not available. Using non-packed folded down convolution. This is 16 times slower per MHz ")


In [ ]:
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from models/resnet18_img_3.onnx


/workspace/qonnx/src/qonnx/core/modelwrapper.py:93: UserWarning: Some old-style domain attributes were automatically converted to new-style,
                i.e. domain=finn to domain=qonnx.custom_op.<general|fpgadataflow|...>
  warnings.warn(


Intermediate outputs will be generated in /workspace/results/KV260_SOM/resnet18_img_3-100-FPS
Final outputs will be generated in build-KV260_SOM/resnet18_img_3-100-FPS
Build log is at build-KV260_SOM/resnet18_img_3-100-FPS/build_dataflow.log
Running step: step_resnet18_tidy [1/13]
Running step: step_resnet18_streamline [2/13]
Running step: step_resnet18_convert_to_hls [3/13]
Running step: step_create_dataflow_partition [4/13]
Running step: step_apply_folding_config [5/13]
Running step: step_generate_estimate_reports [6/13]
Running step: step_hls_codegen [7/13]
Running step: step_hls_ipgen [8/13]
Running step: step_resnet18_set_fifo_depths [9/13]
